In [ ]:
import scipy
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# J = 0.0005 # kg*m^2, rotor inertia
# T_1 = 1/7.66 # gear ratio of link 1
# T_2 = 1/8.846 # gear ratio link 2
# b = 0.1
# T = T_2

def make_default_params():
    params = dict()
    params['J'] = 0.0005
    params['T_1'] = 1/7.66
    params['T_2']  = 1/8.846
    params['b'] = 0.1
    params['T'] = params['T_2']
    
    T = params['T']
    
    # rotor vibrates at this frequency when hit
    # equivalent to a spring
    omega_resonant_hz = 30.0
    omega_resonant_rad_s = 2 * np.pi * 30.0
    gearbox_stiffness_before_reduction = omega_resonant_rad_s**2 * params['J']

    k_s = gearbox_stiffness_before_reduction
    params['k_s'] = k_s
    
    
    # move to reflected parameters
    params['J'] = 1/T**2 * params['J']
    params['k_s'] = 1/T**2 * params['k_s']
    params['b'] = 1/T**2 * params['b']
        
    return params

def linear_system_tf_from_params(params):
    k_s = params['k_s']
    J = params['J']
    b = params['b']
    system = scipy.signal.lti(np.array([k_s]), [J, b, k_s])
    return system


def linear_system_ss_from_params(params):
    """
    Inputs are [tau, x_b]
    """
    k_s = params['k_s']
    J = params['J']
    b = params['b']
    
    A = np.array([[0, 1], [-k_s/J, -b/J]])
    B = np.array([[0,0], [1/J, -k_s/(J)]])
    C = np.array([k_s, 0])
    D = np.array([0, k_s])
    
    sys = scipy.signal.lti(A, B, C, D)
    return sys, [A, B, C, D]

def impedance_ss_system(params, controller_params):
    """
    controller_params is dict with keys ['k_p', 'k_d', 'x_0']
    Note we can only measure \theta, not x. So we implement the feedback using the 
    motor angles
    """
    _, [A,B,C,D] = linear_system_ss_from_params(params)
    
    
    k_p = controller_params['k_p']
    k_d = controller_params['k_d']
    J = params['J']
    # controller gain matrix
    # u = K [\theta, \dot{\theta}]
    K_vec = -1/J * np.array([k_p, k_d])
    K = np.zeros([2,2])
    K[1,:] = K_vec
    A_fb = A + K
    
    k_s = params['k_s']
    B = np.array([[0], [-k_s/J]])
    
    D = D[1]
    
    sys = scipy.signal.lti(A_fb, B, C, D)
    return sys, [A_fb,B,C,D]   
    

default_params = make_default_params()

## Linear Systems Analysis using Bode Plot
Frequency domain analysis

In [ ]:
# make bode plot
# input is "torque desired, i.e. tau = G/T * i", current that you would need to get
# a specific torque out, G is the torque motor constant, T is transformer modulus

system = linear_system_tf_from_params(default_params)

f = np.logspace(0,5)
w = 2 * np.pi * f
w, mag, phase = scipy.signal.bode(system, w)

plt.semilogx(f, mag)
plt.show()

plt.semilogx(f, phase)
plt.show()

params = default_params
print params['k_s']
print params['J']
print params['b']

print default_params

## Implement Impedance controller
Controller is of the form $F =  k_p (x - x_0) + k_d \dot{x}$. Think of $x_0 = 0$ as being nominal
resting length of the leg

In [ ]:
controller_params = dict()
controller_params['k_p'] = 1000
damping_ratio = 0.0
controller_params['k_d'] = 2 * damping_ratio * np.sqrt(controller_params['k_p'])

params = make_default_params()
# params['k_s'] = 30
# params['k_s'] = 100

print "J", params['J']
# params['J'] = params['J']/10.0

# print params['k_s']
# params['k_s'] = 10000 # increase stiffness by factor of 10, markedly improves the tracking
sys, [A,B,C,D] = impedance_ss_system(params, controller_params)

# simulate
dt = 0.01
t_vec = np.arange(0,2, dt)
freq_hz = 1
omega_vec = 2 * t_vec * np.pi * freq_hz
amplitude = 0.1  
x_b = amplitude/2.0 * (1 - np.cos(omega_vec))
x_b_dot = amplitude/2.0 * np.sin(omega_vec)

force_desired = controller_params['k_p']*x_b + controller_params['k_d'] * x_b_dot

# plot the body position (externally specified)
plt.plot(t_vec, x_b)
plt.xlabel('time')
plt.ylabel('x_b')
plt.show()

plt.plot(t_vec, force_desired, 'b')
plt.title("force")
plt.xlabel('time')

plt.show()



# simulate the system
t_out, yout, xout = scipy.signal.lsim(sys, x_b, t_vec)


plt.plot(t_vec, force_desired, 'b')
plt.title("force")
plt.xlabel('time')
plt.plot(t_out, yout, 'r')
plt.show()


## Simulate the response to a step input

Damping turns out to be very important to ensure that the system doesn't oscillate endlessly

In [ ]:
#
output_torque = 100.0
T_vec = np.linspace(0,0.2,100)
N = T_vec.size
params = make_default_params()
sys = linear_system_tf_from_params(params)
T_vec, yout = scipy.signal.step(sys, T=T_vec)
plt.plot(T_vec, yout)
plt.show()



# make sure we can replicate the result using State Space form
sys_ss, _ = linear_system_ss_from_params(params)
U = np.zeros([N, 2])
U[:, 0] = 1
T_vec, yout, xout = scipy.signal.lsim(sys_ss, U, T_vec)
plt.plot(T_vec, yout)


## Backdrivability

In [ ]:
# simulate zero force input with standard system
# make sure we can replicate the result using State Space form
params = make_default_params()
# params['J'] = params['J']/10.0
print params
# params['b'] = params['b']/ 10.0
sys_ss, _ = linear_system_ss_from_params(params)
T_vec = np.linspace(0,1,100)
t_adjust = 0.1
amplitude = 0.1
U = amplitude * np.ones([N, 2])
U[T_vec < t_adjust, 1] = T_vec[T_vec < t_adjust] * amplitude/t_adjust
N = T_vec.size


plt.plot(T_vec, U[:,1])
plt.show()


T_vec, yout, xout = scipy.signal.lsim(sys_ss, U, T_vec)

plt.plot(T_vec, yout)
plt.xlabel("force")
plt.show()

plt.plot(T_vec, U[:,1])
plt.title('x_b')
plt.show()

plt.plot(T_vec, xout[:,0])
plt.title("theta")
print xout.shape


## Simulate using ODE45 instead of linear systems toolbox

This will be needed for doing the non-linear version of things